# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Deepa Rose Thomas
#### Student ID: s3952532

Date: 31-08-2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy


## Introduction
<p style="text-align: justify;">
In this task, We  are required to integrate the given dataset with the solution from the first and second tasks.
There can be many conflicts and issues that can happen while we undertake this process

1. Resolving schema level conflicts and merging data: 
We plan on inspecting and comparing the schema of the first dataset and the second dataset to resolve any conflicts pertaining to the schema level. Since the second data set does not have an implicit Id column, we need to generate a similar column for the third dataset for merging. We have to do a semantic mapping so as to integrate the 2 datasets.
    

2. Resolving data level conflicts 
In this step, we use Pandas libraries so as to resolve duplication in the merged datasets. We also have to identify a global key for the integrated datasets with proper reasoning.

    
3. Finally our output will be the merged dataset
</p>


## .  Defining an Initial Global Schema

There are many conflicts in the two schemas in Dateset1 and Dataset2. 
- Dateset 2 does not have `ID`column
- Dateset 2  does not have annual Salary, instead it has `MonthlyPayment` column.
- Datetime format differs in Detaset 2 and the column names are different.
- `ContractTime` column in Dataset 1 is given as `Type` in Dataset 2
- `ContractYpe` column in Dataset 1 is given as `Full-Time Equivalent (FTE)` in Dataset 2
- `Title` columnn in Dataset 1 is given as `JobTitle` in Dataset 2
- `Company` column in Dataset 1 is `Organization` in Dataset 2
- `Category` has different values in both datasets
- `SourceName` column is missing in Dataset 2.

Hence our schema needs to be matching for both the datasets

##  Import libraries 

In [31]:
# Code to import libraries as you need in this assessment
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### 1. Examining and loading datasets
Examining both the datasets to get an idea regarding the columns and their ordering

In [32]:
# Code to inspect the provided data file
data1 = pd.read_csv('s3952532_dataset1_solution.csv')
data2 = pd.read_csv('s3952532_dataset2.csv')

In [33]:
data1.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70519984,International Marketing Manager / Director,Manchester,A2Z School of English,non specified,permanent,"PR, Advertising & Marketing Jobs",32500.0,2012-05-16 12:00:00,2012-06-15 12:00:00,jobsite.co.uk
1,71857396,"Web Developer HTML, CSS, Wordpress (34 days p...",UK,Recruitment Revolution.Com Limited,non specified,permanent,IT Jobs,20500.0,2013-05-18 15:00:00,2013-06-17 15:00:00,jobsite.co.uk
2,71444746,"Sales Advisor (Events) Swindon covering SN , G...",UK,SMRS Limited new,non specified,permanent,Sales Jobs,34000.0,2013-08-22 12:00:00,2013-10-21 12:00:00,jobsite.co.uk
3,71707588,"Tester, Software Tester, Test Engineer",Leeds,Estio Technology Recruitment Limited,non specified,permanent,IT Jobs,26500.0,2013-07-17 00:00:00,2013-08-16 00:00:00,jobsite.co.uk
4,72679431,Territory Sales / Field Sales Executive,Norwich,Virtuoso Recruitment Limited,non specified,permanent,Sales Jobs,29500.0,2013-07-10 00:00:00,2013-08-09 00:00:00,jobsite.co.uk


Because we have already inspected and corrected the first dataset, we are not checking the types of the attributes in the first dataset

In [34]:
data2.head()

,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [35]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Monthly Payment             5000 non-null   float64
 1   Closing                     5000 non-null   object 
 2   Organisation                4514 non-null   object 
 3   Location                    5000 non-null   object 
 4   Category                    5000 non-null   object 
 5   Full-Time Equivalent (FTE)  5000 non-null   float64
 6   Opening                     5000 non-null   object 
 7   Job Title                   5000 non-null   object 
 8   Type                        3580 non-null   object 
dtypes: float64(2), object(7)
memory usage: 351.7+ KB


***Inference:***

There are inconsistenices in datatypes for this dataset.

In [36]:
print("\nSummary statistics of numeric columns:")
data2.describe()


Summary statistics of numeric columns:


,Monthly Payment,Full-Time Equivalent (FTE)
count,5000.000000,5000.000000
mean,2888.458606,0.983360
std,1347.913223,0.098372
min,417.170000,0.200000
25%,1920.000000,1.000000
50%,2605.000000,1.000000
75%,3541.670000,1.000000
max,8000.000000,1.000000


### 2. Resolving schema conflicts
In this section we resolve the conflicts related to the schema of the datasets

#### Conflict 1: Naming Conventions
The columns in both the datasets have different names and appears to be in different order. 
- As an initial step, I plan on renaming the columns in the second dataset to match the first.

In [37]:
data1.columns

Index(['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'],
      dtype='object')

In [38]:
data2.columns

Index(['Monthly Payment', 'Closing', 'Organisation', 'Location', 'Category',
       'Full-Time Equivalent (FTE)', 'Opening', 'Job Title', 'Type'],
      dtype='object')

In [39]:
#1. RENAMING COLUMNS
columns_mapping = {
    'Organisation': 'Company',
    'Type': 'ContractTime',
    'Monthly Payment': 'Salary',
    'Closing': 'CloseDate',
    'Location': 'Location',
    'Category': 'Category',
    'Opening': 'OpenDate',
    'Job Title': 'Title',
    'Full-Time Equivalent (FTE)': 'ContractType'
}
data2.rename(columns=columns_mapping, inplace=True)
    

In [40]:
data2.columns

Index(['Salary', 'CloseDate', 'Company', 'Location', 'Category',
       'ContractType', 'OpenDate', 'Title', 'ContractTime'],
      dtype='object')

In [41]:
data2.head()

,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


#### Conflict 2 : Inconsistency in column values

The Salary column in dataset 1 is annual salaries, where as in dataset 2, they are monthly salaries. We choose to convert the monthly salaries to annual salaries in the second dataset for consistency in column values


In [42]:
data2['Salary'] = data2['Salary']*12

In [43]:
data2['Salary'].head()

0    14400.00
1    37500.00
2    51999.96
3    45000.00
4    35000.04
Name: Salary, dtype: float64

**Conflict 3: Datatype mismatch**

There are columns which does not have the required datatypes. We need to change them to the right datatype

In [44]:
# Convert date columns to datetime format
date_columns = ['CloseDate', 'OpenDate']
for column in date_columns:
    data2[column] = pd.to_datetime(data2[column], dayfirst=True)

In [45]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Salary        5000 non-null   float64       
 1   CloseDate     5000 non-null   datetime64[ns]
 2   Company       4514 non-null   object        
 3   Location      5000 non-null   object        
 4   Category      5000 non-null   object        
 5   ContractType  5000 non-null   float64       
 6   OpenDate      5000 non-null   datetime64[ns]
 7   Title         5000 non-null   object        
 8   ContractTime  3580 non-null   object        
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 351.7+ KB


***Inference:*** 
The datatype of OpenDate and CloseDate columns have been corrected

**Conflict 4 : Missing ID column in Dataset 2**

- We do not have an Id column in the second dataset. Hence as the first step, we need to generate random unique Ids matching the ones in the first data set


In [46]:
#function to create unique IDs for 2nd dataset
def generate_unique_ids(existing_ids):
    new_id = np.random.randint(10000000, 99999999)  # Generate a random 8-digit integer
    while new_id in existing_ids:
        new_id = np.random.randint(10000000, 99999999)  # Keep generating until unique
    return new_id

existing_ids = data1['Id'].tolist()  # Existing IDs in first data set
data2['Id'] = data2.apply(lambda row: generate_unique_ids(existing_ids), axis=1)

In [47]:
data2.head()

,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime,Id
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN,39543361
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,48425884
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,34161114
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN,66180101
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN,89435195


In [48]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Salary        5000 non-null   float64       
 1   CloseDate     5000 non-null   datetime64[ns]
 2   Company       4514 non-null   object        
 3   Location      5000 non-null   object        
 4   Category      5000 non-null   object        
 5   ContractType  5000 non-null   float64       
 6   OpenDate      5000 non-null   datetime64[ns]
 7   Title         5000 non-null   object        
 8   ContractTime  3580 non-null   object        
 9   Id            5000 non-null   int64         
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 390.8+ KB


***Inference:***
- Id column has been created
- Datatypes have been corrected
- Columns are renamed

**Conflict 4 : Incorrect Values(-,N/A, None) in Columns and Order of Columns**
- The Null and undesirable values has be to changed to non-specified
- Contract Time value has to be changed to full-time, part-time or non-specified
- Source Column has to be created with value as 'www.jobhuntlisting.com'
- Category Column has to be mapped into the required categories as per the first dataset
- The ordering of the columns also appear to be different, hence for ease of understanding and clarity, I plan on making the column arranged in the same order in both data sets

In [49]:
# Map values in 'ContractTime' column to desired values
contract_time_mapping = {1: 'full_time', 0: 'non-specified'}
# Update values in 'ContractTime' column based on the mapping
data2['ContractType'] = data2['ContractType'].map(contract_time_mapping)
# Replace values greater than 0 and less than 1 with 'part_time'
data2.loc[(data2['ContractType'] != 'full_time') & (data2['ContractType'] != 'non-specified'), 'ContractType'] = 'part_time'

In [50]:
# Verify the changes
print("Updated dataset with modified 'ContractType' column:")
data2.head()

Updated dataset with modified 'ContractType' column:


,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime,Id
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Education,part_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN,39543361
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,48425884
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,34161114
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN,66180101
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,full_time,2012-12-29 12:00:00,Channel Account Manager Software,NaN,89435195


In [51]:
# Replace empty values in ContractTime and Company columns with 'non-specified'
data2['ContractTime'].fillna('non-specified', inplace=True)
data2['Company'].fillna('non-specified', inplace=True)

In [52]:
# Add a new column 'SourceName' to df1 and fill with 'non-specified'
data2['SourceName'] = 'www.jobhuntlisting.com'

In [53]:
data2.head()

,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime,Id,SourceName
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Education,part_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),non-specified,39543361,www.jobhuntlisting.com
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,48425884,www.jobhuntlisting.com
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,34161114,www.jobhuntlisting.com
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",non-specified,66180101,www.jobhuntlisting.com
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,full_time,2012-12-29 12:00:00,Channel Account Manager Software,non-specified,89435195,www.jobhuntlisting.com


In [54]:
data2.isna().sum()

Salary          0
CloseDate       0
Company         0
Location        0
Category        0
ContractType    0
OpenDate        0
Title           0
ContractTime    0
Id              0
SourceName      0
dtype: int64

In [55]:
data1.isna().sum()

Id              0
Title           0
Location        0
Company         0
ContractType    0
ContractTime    0
Category        0
Salary          0
OpenDate        0
CloseDate       0
SourceName      0
dtype: int64

In [56]:
#inspecting Category Column
data2['Category'].unique()

array(['Education', 'Information Technology', 'Sales', 'Finance',
       'Engineering', 'Health', 'Hospitality', 'Marketing'], dtype=object)

In [57]:
data1['Category'].unique()

array(['PR, Advertising & Marketing Jobs', 'IT Jobs', 'Sales Jobs',
       'Engineering Jobs', 'Accounting & Finance Jobs',
       'Healthcare & Nursing Jobs', 'Teaching Jobs',
       'Hospitality & Catering Jobs'], dtype=object)

In [58]:
column_mapping = {
    'Information Technology' : 'IT Jobs',
    'Sales': 'Sales Jobs',
    'Engineering': 'Engineering Jobs',
    'Finance': 'Accounting & Finance Jobs',
    'Marketing' : 'PR, Advertising & Marketing Jobs',
    'Hospitality' : 'Hospitality & Catering Jobs',
    'Education' : 'Teaching Jobs',
    'Health' : 'Healthcare & Nursing Jobs'
}

In [59]:
data2['Category'] = data2['Category'].map(column_mapping)

In [60]:
data2['Category'].unique()

array(['Teaching Jobs', 'IT Jobs', 'Sales Jobs',
       'Accounting & Finance Jobs', 'Engineering Jobs',
       'Healthcare & Nursing Jobs', 'Hospitality & Catering Jobs',
       'PR, Advertising & Marketing Jobs'], dtype=object)

In [61]:
data2.head()

,Salary,CloseDate,Company,Location,Category,ContractType,OpenDate,Title,ContractTime,Id,SourceName
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Teaching Jobs,part_time,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),non-specified,39543361,www.jobhuntlisting.com
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,IT Jobs,full_time,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,48425884,www.jobhuntlisting.com
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,IT Jobs,full_time,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,34161114,www.jobhuntlisting.com
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales Jobs,full_time,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",non-specified,66180101,www.jobhuntlisting.com
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales Jobs,full_time,2012-12-29 12:00:00,Channel Account Manager Software,non-specified,89435195,www.jobhuntlisting.com


In [62]:
new_order = ['Id','Title','Location','Company','ContractType','ContractTime','Category','Salary','OpenDate','CloseDate','SourceName']
data2 = data2[new_order]

In [63]:
data2.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,39543361,Higher Level Teaching Assistant (HLTA),UK,Protocol Education,part_time,non-specified,Teaching Jobs,14400.00,2012-11-27 12:00:00,2012-12-27 12:00:00,www.jobhuntlisting.com
1,48425884,Software Developers / Software Architects Not...,UK,Cordius Ltd,full_time,Permanent,IT Jobs,37500.00,2013-06-26 15:00:00,2013-08-25 15:00:00,www.jobhuntlisting.com
2,34161114,Senior BI Consultant (Cognos),Walton-On-Thames,Understanding Recruitment,full_time,Permanent,IT Jobs,51999.96,2012-10-18 15:00:00,2013-01-16 15:00:00,www.jobhuntlisting.com
3,66180101,"Technical Sales Manager, Pigments Fillers to S...",UK,Wallace Hind Selection,full_time,non-specified,Sales Jobs,45000.00,2012-12-15 15:00:00,2013-02-13 15:00:00,www.jobhuntlisting.com
4,89435195,Channel Account Manager Software,Eastern England,Greenhill Group Ltd,full_time,non-specified,Sales Jobs,35000.04,2012-12-29 12:00:00,2013-01-12 12:00:00,www.jobhuntlisting.com


In [64]:
data1.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70519984,International Marketing Manager / Director,Manchester,A2Z School of English,non specified,permanent,"PR, Advertising & Marketing Jobs",32500.0,2012-05-16 12:00:00,2012-06-15 12:00:00,jobsite.co.uk
1,71857396,"Web Developer HTML, CSS, Wordpress (34 days p...",UK,Recruitment Revolution.Com Limited,non specified,permanent,IT Jobs,20500.0,2013-05-18 15:00:00,2013-06-17 15:00:00,jobsite.co.uk
2,71444746,"Sales Advisor (Events) Swindon covering SN , G...",UK,SMRS Limited new,non specified,permanent,Sales Jobs,34000.0,2013-08-22 12:00:00,2013-10-21 12:00:00,jobsite.co.uk
3,71707588,"Tester, Software Tester, Test Engineer",Leeds,Estio Technology Recruitment Limited,non specified,permanent,IT Jobs,26500.0,2013-07-17 00:00:00,2013-08-16 00:00:00,jobsite.co.uk
4,72679431,Territory Sales / Field Sales Executive,Norwich,Virtuoso Recruitment Limited,non specified,permanent,Sales Jobs,29500.0,2013-07-10 00:00:00,2013-08-09 00:00:00,jobsite.co.uk


***Inference***
- Null Values Handled
- Contract Time and Contract type columns have the required values
- Source Column has been inserted
- Category Column values have been corrected as per dataset1
- Columns have been reordered as per the dataset 1

### 3. Merging data

We can now merge the two datasets into one

In [65]:

# Concatenate the two data frames vertically
merged_data = pd.concat([data1, data2],ignore_index = True)

# Display the unified table
print("Unified table after Merging:")
merged_data.head()

Unified table after Merging:


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70519984,International Marketing Manager / Director,Manchester,A2Z School of English,non specified,permanent,"PR, Advertising & Marketing Jobs",32500.0,2012-05-16 12:00:00,2012-06-15 12:00:00,jobsite.co.uk
1,71857396,"Web Developer HTML, CSS, Wordpress (34 days p...",UK,Recruitment Revolution.Com Limited,non specified,permanent,IT Jobs,20500.0,2013-05-18 15:00:00,2013-06-17 15:00:00,jobsite.co.uk
2,71444746,"Sales Advisor (Events) Swindon covering SN , G...",UK,SMRS Limited new,non specified,permanent,Sales Jobs,34000.0,2013-08-22 12:00:00,2013-10-21 12:00:00,jobsite.co.uk
3,71707588,"Tester, Software Tester, Test Engineer",Leeds,Estio Technology Recruitment Limited,non specified,permanent,IT Jobs,26500.0,2013-07-17 00:00:00,2013-08-16 00:00:00,jobsite.co.uk
4,72679431,Territory Sales / Field Sales Executive,Norwich,Virtuoso Recruitment Limited,non specified,permanent,Sales Jobs,29500.0,2013-07-10 00:00:00,2013-08-09 00:00:00,jobsite.co.uk


In [66]:
merged_data.shape

(55745, 11)

In [67]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55745 entries, 0 to 55744
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            55745 non-null  int64  
 1   Title         55745 non-null  object 
 2   Location      55745 non-null  object 
 3   Company       55745 non-null  object 
 4   ContractType  55745 non-null  object 
 5   ContractTime  55745 non-null  object 
 6   Category      55745 non-null  object 
 7   Salary        55745 non-null  float64
 8   OpenDate      55745 non-null  object 
 9   CloseDate     55745 non-null  object 
 10  SourceName    55745 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.7+ MB


### 4. Resolving data conflicts:
We need to check for certain data conflicts after we have the merged dataframe. 


#### Conflict 1: Duplicate rows
When we integrate multiple dataframes together, there might be duplicate records that we need to further deal with. 
Hence we examine if there are duplicate rows in our merged dataframe

In [68]:
duplicates = merged_data[merged_data.duplicated(keep=False)] # showing all duplicated records
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)


There are 0 duplicate records found


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


In [69]:
#Check for duplicate IDs
duplicate_ids = merged_data.index[merged_data.index.duplicated()]
if len(duplicate_ids) == 0:
    print("No duplicate IDs found.")
else:
    print("Duplicate IDs found in unified_df:", duplicate_ids)


No duplicate IDs found.


***Inference:***

Here we do not have any duplicate records or duplicate Ids in our final merged data frame. Hence we dont have to handle the columns

**Conflict 2 : Missing Values**

In [70]:
# Code to inspect data and identify conflicts in data
merged_data.isna().sum()

Id              0
Title           0
Location        0
Company         0
ContractType    0
ContractTime    0
Category        0
Salary          0
OpenDate        0
CloseDate       0
SourceName      0
dtype: int64

In [71]:
null_counts = merged_data.isnull().sum()

# Display columns with null values and their counts
columns_with_nulls = null_counts[null_counts > 0]
if not columns_with_nulls.empty:
    print("Columns with null values and their counts:")
    print(columns_with_nulls)
else:
    print("No null values in unified_df.")

No null values in unified_df.


***Inferene***
- Null values and Duplicate values are inspected and were found to be absent

In [73]:
# Convert OpenDate and CloseDate columns to datetime objects
merged_data['OpenDate'] = pd.to_datetime(merged_data['OpenDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
merged_data['CloseDate'] = pd.to_datetime(merged_data['CloseDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [74]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55745 entries, 0 to 55744
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            55745 non-null  int64         
 1   Title         55745 non-null  object        
 2   Location      55745 non-null  object        
 3   Company       55745 non-null  object        
 4   ContractType  55745 non-null  object        
 5   ContractTime  55745 non-null  object        
 6   Category      55745 non-null  object        
 7   Salary        55745 non-null  float64       
 8   OpenDate      55745 non-null  datetime64[ns]
 9   CloseDate     55745 non-null  datetime64[ns]
 10  SourceName    55745 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 4.7+ MB


In [76]:
# Identify rows where OpenDate > CloseDate
inconsistent_dates = merged_data[merged_data['OpenDate'] > merged_data['CloseDate']]
inconsistent_dates.head(10)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


***Inference***: Date columns is handled

#### Finding global key for the data


In [75]:
# Code to identify global key
# Check for duplicates based on the 'Id' column
duplicate_ids = merged_data[merged_data.duplicated('Id', keep=False)]

# Print the duplicate Ids and their occurrences
print(duplicate_ids['Id'].value_counts())

# Verify uniqueness of 'Id' column
is_unique = merged_data['Id'].is_unique
print(f"Is 'Id' column unique: {is_unique}")

Series([], Name: Id, dtype: int64)
Is 'Id' column unique: True


The `Id` column is chosen as the unique key for the final integrated job data due to its:

* *Unique*: Each 'Id' is a unique identifier for job ads, assigned by the source, there by ensuring uniqueness.
* *Consistent*: 'Id' is consistent across datasets which means we can track the job ads properly.
* *Reliable*: Since Id a standardized identifier, 'Id' is stable across changes.
* *Ease of Identification*: Numeric or alphanumeric 'Id' provide clear ad differentiation.
* *Efficiency in Merging*: 'Id' simplifies merging, thereby reducing the risks of data loss or duplication.

### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- '\<student\_id\>_dataset_integrated.csv'

In [77]:
# code to save output data
merged_data.to_csv('s3952532_dataset_integrated.csv')

## Summary of the Assessment Task
In this task we have tried to merge two datasets.

While merging two datasets, we need to keep in mind regarding the schema level mis matches and conflicts that might occur. Here the conflicts included dissimilar column names, difference in the order of columns, missing columns, inconsistent datatypes of columns etc. 
Before merging the datasets, we handled these inconsistencies.

Once the dataset is merged, we further checked for data level conflicts which included duplicate rows and missing values.
Later we identified a column (Id) which can identify all the columns uniquely and consistently.
Finally we saved the merged dataframe.
